In [1]:
import labelbox as lb
import urllib.request
import re

API_KEY = "XXXX"

PROJECT_ID = "YYYY"

In [2]:
def constructIOB(articleName, startIndicies, endIndices, labeledWord, labeledEntity,labeledSubentity, text):
    sorted_indices = sorted(range(len(startIndicies)), key=lambda i: startIndicies[i])
    startIndicies = [startIndicies[i] for i in sorted_indices]
    endIndices = [endIndices[i] for i in sorted_indices]
    labeledWord = [labeledWord[i] for i in sorted_indices]
    labeledEntity = [labeledEntity[i] for i in sorted_indices]
    labeledSubentity = [labeledSubentity[i] for i in sorted_indices]

    i = 0
    count = 0
    word = ""
    IOB = ""
    result = ""

    speicalCharacters = r'[!@#$%^&*()-_+{}\[\]:;<>,،.?~\\]'
    while i < len(text):
        if count < len(startIndicies) and i == startIndicies[count]:
            i = endIndices[count] + 1
            word = labeledWord[count]
            if (labeledEntity[count] != "Date" and labeledEntity[count] != "Nationality" and labeledEntity[count] != "Currency"):
                IOB += labeledSubentity[count] + " " + word + "\n"
            else:
                IOB += labeledEntity[count] + " " + word + "\n"
            count += 1
            word = ""
        elif text[i] != " " and i != len(text) - 1:
            word += text[i]
            i += 1

        elif text[i] == " " or i == len(text) - 1:
            word = word
            word = word.replace("\n O\n", "\n")
            IOB += "OUTSIDE " + word + "\n"
            word = ""
            i += 1
        else:

            i += 1


    # print(IOB)

    IOB = IOB.replace("\n OUTSIDE\n", "\n")
    IOB = IOB.replace(",", "")
    IOB = IOB.replace("،","")
    IOB = IOB.replace("؛","")
    IOB = IOB.replace("(","")
    IOB = IOB.replace(")","")
    IOB = IOB.replace("[","")
    IOB = IOB.replace("]","")
    IOB = IOB.replace("{","")
    IOB = IOB.replace("}","")
    IOB = IOB.replace(":","")
    IOB = IOB.replace("«","")
    IOB = IOB.replace("»","")
    # IOB = IOB.replace(".", "")
    IOB = IOB.replace("-", " ")

    lines = IOB.split('\n')
    # print(IOB)

    cleanIOB = ""
    index = 0

    # print(lines)

    fixedIOB = ""

    for line in lines:
        curr_line = line.split()
        if len(curr_line) > 1:
            fixedIOB += line + "\n"
        elif len(curr_line) == 1:
            arabic_pattern = re.compile(r'^[0-9\u0600-\u06FF\s]+$')
            if arabic_pattern.match(curr_line[0]):
                fixedIOB += "OUTSIDE " + curr_line[0] + "\n"

    # print(fixedIOB)

    lines = fixedIOB.split('\n')

    adjustedIOB = ""

    sentence_termination_characters = ['.', '?', '!']

    for line in lines:
        curr_line = line.split()

        if len(curr_line) > 1:
            subentity = curr_line[0]
            words = curr_line[1:]

            words = [re.sub(r'[^\u0600-\u06FF0-9.?!]', '', word) for word in words]
    #         print(words)
            adjustedIOB += subentity + " "

            for word in words:

                if re.match(r'^\d+(\.\d+)?$', word):
                    adjustedIOB += word + " "
                    continue

                if not any(char in sentence_termination_characters for char in word):
                    adjustedIOB += word + " "
                    continue
                elif any(char in sentence_termination_characters for char in word):
                    for character in word:

                        if character not in sentence_termination_characters:
                            adjustedIOB += character
                            continue
                        if character in sentence_termination_characters:
                            adjustedIOB += "\nOUTSIDE " + character + "\n"

            adjustedIOB += "\n"
    #     elif len(curr_line) == 1:
    #         adjustedIOB += "OUTSIDE " + curr_line[0] 



    # print(adjustedIOB)
    cleanedIOB = ""

    for line in adjustedIOB.split('\n'):
        curr_line = line.split()
        if len(curr_line) >= 2:
            for item in curr_line:
                cleanedIOB += item + " "
            cleanedIOB += "\n"
        elif len(curr_line) == 1:
            arabic_pattern = re.compile(r'^[0-9\u0600-\u06FF\s]+$')
            if arabic_pattern.match(curr_line[0]):
                cleanedIOB += "OUTSIDE " + curr_line[0] + "\n"

    # print(cleanedIOB)

    lines = cleanedIOB.split('\n')          
    # print(lines)         

    for line in range(0, len(lines)-1):
        currentLine = lines[line].split(" ")    
        tokens = [token.strip() for token in currentLine[1:] if token.strip()]
        for i in range(0, len(tokens)):
            if i == 0 and currentLine[0] != "OUTSIDE":
                result += "B-" + currentLine[0] + " " + tokens[i] + "\n"
            elif i != 0 and currentLine[0] != "OUTSIDE":
                result += "I-" + currentLine[0] + " " + tokens[i] + "\n"
            else:
                result += currentLine[0] + " " + tokens[i] + "\n"

    result += "OUTSIDE " + text[-1]

    # print(result)

    final = ""

    lines = result.split('\n')

    for line in lines:
        curr_line = line.split()

        if len(curr_line) == 2 and curr_line[0] in tags:
            final += line + "\n"

    # print(final)
    
    with open("IOB/" + articleName, 'w') as file:
        file.write(final)
#     print(result)
    
    return final


In [5]:
client = lb.Client(API_KEY)
project = client.get_project(PROJECT_ID)
task = project.export_v2()
task.wait_till_done()

labeledWord = []
labeledEntity = []
labeledSubentity = []
startIndicies = []
endIndices = []

word = ""

output2024 = ""

count = 0;
artCount = 0
if task.errors:
    print(task.errors)
else:

    for export_entry in task.result:
        
        name = export_entry['data_row']['external_id']
        
#         if name == "Article1840.txt":
        # Filter on entries with a label
        if export_entry["projects"][PROJECT_ID]["labels"]:

            data_row_id = export_entry["data_row"]["id"]

            articleName = export_entry['data_row']['external_id']

            print("Data Row Name:", articleName)



            # Download the asset
            response = urllib.request.urlopen(export_entry["data_row"]["row_data"])
            text = response.read().decode("utf-8")

            labeledWord.clear();
            labeledEntity.clear();
            labeledSubentity.clear();
            startIndicies.clear();
            endIndices.clear();

            word = ""
            count = 0
            i = 0
            numberOfSubentities = 0
#             if articleName == "Article1841.txt":

            if len(export_entry["projects"][PROJECT_ID]["labels"]) == 2:
                label = export_entry["projects"][PROJECT_ID]["labels"][0]
            elif len(export_entry["projects"][PROJECT_ID]["labels"]) == 1:
                label = export_entry["projects"][PROJECT_ID]["labels"][0]
            else:
                print("\n\nERROR ERROR ERROR\n\n")

            artCount += 1
            label_id = label["id"]
#                     if label_id == "clsrv204t4oi507dz60johy7n" or label_id == "clsrv474j52bk07g4bxouhgzc" or label_id == "clsrxszhm52w507fa2tsc6nx5" or label_id == "clst66mera69w07duelxreerq" or label_id == "clsta7gh5affs07g47lgl0d8y" or label_id == "clstbpsz9akfl07edcgyq8h6p" or label_id == "clsug6lmm2jsq07hs2fry5ahj" or label_id == "clsug6ki70p9607gi66f419y4" or label_id == "clsumulp60d0507umcmpd2wgz" or label_id == "clsujyw1m364l07hb9lzchoz1" or label_id == "clsvddl6x0kn507dm7wbuaje0" or label_id == "clsug80op2k7907isdzfcccyf" or label_id == "clfkflf9o043r07tdcp3obif2" or label_id == "clsvrmmzm10h307hlb072gb5j" or label_id == "clsvrmo3n10h707hlgbpy8hft" or label_id == "clsvtfrn90ico07fnbjhe2k1w" or label_id == "clsvurzm4020w07fm9sl5b8yo" or label_id == "clsvwcxoj08fr07du378kf4vu" or label_id == "clsvy4z5v0azy07fmhbr2270y" or label_id == "clsw09ki00dyj07cig94lemqe" or label_id == "clswaymfw028t07iq7i8838kz":
            print(label_id)
            for annotation in label["annotations"]["objects"]:
                if annotation["annotation_kind"] == "TextEntity":
                    feature_schema_id = annotation["feature_schema_id"]
                    annotation_name = annotation["name"]
                    start = annotation["location"]["start"]
                    end = annotation["location"]["end"]
                    if (
                        annotation_name == "Person"
                        or annotation_name == "Organization"
                        or annotation_name == "Location"
                        or annotation_name == "Product"
                        or annotation_name == "Art"
                        or annotation_name == "Event"
                        or annotation_name == "Building"
                        or annotation_name == "Other"
                    ):
                        subentity = annotation["classifications"][0]["radio_answer"]["name"]
                        numberOfSubentities += 1
                    else:
                        subentity = ""
                    labeled_text = text[start : end + 1]
                    labeledWord.append(labeled_text)
                    labeledEntity.append(annotation_name)
                    labeledSubentity.append(subentity)
                    startIndicies.append(start)
                    endIndices.append(end)

            debug = constructIOB(articleName, startIndicies, endIndices, labeledWord, labeledEntity,labeledSubentity, text)
            print(debug)
            if artCount >= 1:
                break
                

Data Row Name: Article1000.txt
clfkflf9o043r07tdcp3obif2
B-Company جوجل
OUTSIDE أو
B-Company غوغل
OUTSIDE أو
B-Company قوقل
B-Language بالإنجليزية
OUTSIDE هي
OUTSIDE شركة
B-Nationality أمريكية
OUTSIDE عامة
OUTSIDE متخصصة
OUTSIDE في
OUTSIDE مجال
OUTSIDE الإعلان
OUTSIDE المرتبط
OUTSIDE بخدمات
OUTSIDE البحث
OUTSIDE على
OUTSIDE الإنترنت
OUTSIDE وإرسال
OUTSIDE رسائل
OUTSIDE بريد
OUTSIDE إلكتروني
OUTSIDE عن
OUTSIDE طريق
OUTSIDE جي
OUTSIDE ميل
OUTSIDE .
OUTSIDE واختير
OUTSIDE اسم
B-Company جوجل
OUTSIDE الذي
OUTSIDE يعكس
OUTSIDE المُهمة
OUTSIDE التي
OUTSIDE تقوم
OUTSIDE بها
OUTSIDE الشركة
OUTSIDE وهي
OUTSIDE تنظيم
OUTSIDE ذلك
OUTSIDE الكم
OUTSIDE الهائل
OUTSIDE من
OUTSIDE المعلومات
OUTSIDE المُتاحة
OUTSIDE على
OUTSIDE الويب
OUTSIDE .
OUTSIDE يضاف
OUTSIDE إلى
OUTSIDE ذلك
OUTSIDE توفيرها
OUTSIDE لإمكانية
OUTSIDE نشر
OUTSIDE المواقع
OUTSIDE التي
OUTSIDE توفر
OUTSIDE معلومات
OUTSIDE نصية
OUTSIDE ورسومية
OUTSIDE في
OUTSIDE شكل
OUTSIDE قواعد
OUTSIDE بيانات
OUTSIDE وخرائط
OUTSIDE على
OUTSIDE شبكة
OUT

In [3]:
tags = [
    'B-Actor', 'B-Architect', 'B-Artist', 'B-Athlete', 'B-Author', 'B-Coach', 'B-Director', 'B-Doctor', 'B-Engineer',
    'B-Monarch', 'B-Musician', 'B-Politician', 'B-Religious_Leader', 'B-Soldier', 'B-Terrorist', 'B-Airline', 
    'B-Company', 'B-Educational_Institution', 'B-Fraternity_Sorority', 'B-Sports_League', 'B-Sports_Team', 
    'B-Terrorist_Organization', 'B-Government_Agency', 'B-Government', 'B-Political_Party', 'B-Educational_Department', 
    'B-Military', 'B-News_Agency', 'B-City', 'B-Country', 'B-County', 'B-Province', 'B-Railway', 'B-Road', 'B-Bridge', 
    'B-Body_Of_Water', 'B-Island', 'B-Mountain', 'B-Glacier', 'B-Astral_Body', 'B-Cemetery', 'B-Park', 'B-Engine', 
    'B-Airplane', 'B-Car', 'B-Ship', 'B-Spacecraft', 'B-Train', 'B-Camera', 'B-Mobile_Phone', 'B-Computer', 'B-Software', 
    'B-Game', 'B-Instrument', 'B-Weapon', 'B-Film', 'B-Play', 'B-Written_Work', 'B-Newspaper', 'B-Music', 'B-Attack', 
    'B-Election', 'B-Protest', 'B-Military_Conflict', 'B-Natural_Disaster', 'B-Sports_Event', 'B-Terrorist_Attack', 
    'B-Airport', 'B-Dam', 'B-Hospital', 'B-Hotel', 'B-Library', 'B-Power_Station', 'B-Restaurant', 'B-Sports_Facility', 
    'B-Theater', 'B-Time', 'B-Color', 'B-Award', 'B-Educational_Degree', 'B-Title', 'B-Law', 'B-Ethnicity', 'B-Language', 
    'B-Religion', 'B-God', 'B-Chemical_Thing', 'B-Biological_Thing', 'B-Medical_Thing', 'B-Disease', 'B-Symptom', 'B-Drug', 
    'B-Body_Part', 'B-Living_Thing', 'B-Animal', 'B-Food', 'B-Website', 'B-Broadcast_Network', 'B-Broadcast_Program', 
    'B-Tv_Channel', 'B-Currency', 'B-Stock_Exchange', 'B-Algorithm', 'B-Programming_Language', 'B-Transit_System', 
    'B-Transit_Line', 'B-Nationality', 'B-Date', 'I-Actor', 'I-Architect', 'I-Artist', 'I-Athlete', 'I-Author', 'I-Coach', 
    'I-Director', 'I-Doctor', 'I-Engineer', 'I-Monarch', 'I-Musician', 'I-Politician', 'I-Religious_Leader', 'I-Soldier', 
    'I-Terrorist', 'I-Airline', 'I-Company', 'I-Educational_Institution', 'I-Fraternity_Sorority', 'I-Sports_League', 
    'I-Sports_Team', 'I-Terrorist_Organization', 'I-Government_Agency', 'I-Government', 'I-Political_Party', 'I-Educational_Department', 
    'I-Military', 'I-News_Agency', 'I-City', 'I-Country', 'I-County', 'I-Province', 'I-Railway', 'I-Road', 'I-Bridge', 
    'I-Body_Of_Water', 'I-Island', 'I-Mountain', 'I-Glacier', 'I-Astral_Body', 'I-Cemetery', 'I-Park', 'I-Engine', 
    'I-Airplane', 'I-Car', 'I-Ship', 'I-Spacecraft', 'I-Train', 'I-Camera', 'I-Mobile_Phone', 'I-Computer', 'I-Software', 
    'I-Game', 'I-Instrument', 'I-Weapon', 'I-Film', 'I-Play', 'I-Written_Work', 'I-Newspaper', 'I-Music', 'I-Attack', 
    'I-Election', 'I-Protest', 'I-Military_Conflict', 'I-Natural_Disaster', 'I-Sports_Event', 'I-Terrorist_Attack', 
    'I-Airport', 'I-Dam', 'I-Hospital', 'I-Hotel', 'I-Library', 'I-Power_Station', 'I-Restaurant', 'I-Sports_Facility', 
    'I-Theater', 'I-Time', 'I-Color', 'I-Award', 'I-Educational_Degree', 'I-Title', 'I-Law', 'I-Ethnicity', 'I-Language', 
    'I-Religion', 'I-God', 'I-Chemical_Thing', 'I-Biological_Thing', 'I-Medical_Thing', 'I-Disease', 'I-Symptom', 'I-Drug', 
    'I-Body_Part', 'I-Living_Thing', 'I-Animal', 'I-Food', 'I-Website', 'I-Broadcast_Network', 'I-Broadcast_Program', 
    'I-Tv_Channel', 'I-Currency', 'I-Stock_Exchange', 'I-Algorithm', 'I-Programming_Language', 'I-Transit_System', 
    'I-Transit_Line', 'I-Nationality', 'I-Date', 'OUTSIDE'
]